In [21]:
import pandas as pd
import numpy as np
import re
import psycopg2
# Database credentials
user = "cspoms"
password = "cspoms"
host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
port = "5432"
database = "CSPODB"
sslmode = "verify-ca"
sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

# Initialize cursor and connection to None
connection = None
cursor = None


# Establish the connection
connection = psycopg2.connect(
    database=database,
    user=user,
    password=password,
    host=host,
    port=port,
    sslmode=sslmode,
    sslcert=sslcert,
    sslkey=sslkey,
    sslrootcert=sslrootcert
)
post_cur = connection.cursor()

In [22]:
# Execute the query
post_cur.execute('''SELECT * FROM xxpo_orcl2cspoms_vend_stg 
WHERE request_id = '593' 
AND
(vendor_foodlink_flg in ('1','2')
or vendor_freight_flg = 'Y'
or bic_vendor_facility in (14 ,21))
''')

# Fetch all the rows from the executed query
rows = post_cur.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in post_cur.description]

# Convert the fetched data into a DataFrame
clone_df = pd.DataFrame(rows, columns=columns)
clone_df

,stg_seq_id,bic_vendor_facility,bic_vendor_number,bic_buyer_number,bic_crp_level,bic_status,bic_po_recommended_ind,bic_afe_type1,bic_afe_fax_number,bic_afe_contact,...,file_name,remark1_flag,remark1,remark2_flag,remark2,remark3_flag,remark3,remark4_flag,remark4,home_destination
0,350300,14,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
1,348098,14,69910,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
2,350686,14,611997,FF,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
3,348124,14,70488,JJ,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
4,347129,14,21935,LY,,A,F,None,513 936 2764,FAX NUMBER,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8684,350062,14,420676,HA,,A,F,None,214 421 3292,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
8685,348026,14,68454,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
8686,346507,14,8622,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
8687,346320,14,5797,DV,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None


In [24]:
print(f'input_df that should be clone for all orgs',clone_df)
all_active_orgs = "SELECT facility_code FROM cspoms_facility_master WHERE inactive_date IS NULL"
post_cur.execute(all_active_orgs)
# Fetch all results into a list
all_active_orgs_list = post_cur.fetchall()
# Extract facility_code values from the result list
facility_codes_list = [int(row[0]) for row in all_active_orgs_list]
print(len(facility_codes_list),facility_codes_list)

input_df that should be clone for all orgs       stg_seq_id  bic_vendor_facility bic_vendor_number bic_buyer_number  \
0         350300                   14            494951               L5   
1         348098                   14             69910               ZZ   
2         350686                   14            611997               FF   
3         348124                   14             70488               JJ   
4         347129                   14             21935               LY   
...          ...                  ...               ...              ...   
8684      350062                   14            420676               HA   
8685      348026                   14             68454               ZZ   
8686      346507                   14              8622               ZZ   
8687      346320                   14              5797               DV   
8688      348105                   14             69952               ZZ   

     bic_crp_level bic_status bic_po_recomme

In [31]:
def clone_rows_active_vendor_facilty2( input_df, column_to_change, new_values):
    """
    Clone rows in a DataFrame with Facility changed based on a list of values.
    Skip cloning if the new value matches the original row's condition.
    Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame.
    - column_to_change (str): The name of the column to be changed.
    - new_values (list): The list of new values for the specified column.

    Returns:
    - pd.DataFrame: The resulting DataFrame with cloned rows and the specified column changed.
    """
    new_rows_list = []  # Use a list to collect new rows

    for index, row in input_df.iterrows():
        bic_vendor_facility_value = row[column_to_change]
        # Clone rows only if the new value is different from the original value
        for new_value in new_values:
            if new_value != bic_vendor_facility_value:
                new_row = row.copy()
                new_row[column_to_change] = new_value
                new_row['home_destination'] = bic_vendor_facility_value
                new_rows_list.append(new_row)

    # Convert the list of new rows to a DataFrame
    new_rows = pd.DataFrame(new_rows_list)

    # Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame
    new_rows['bic_vendor_type'] = 'C'
    print('new rows to clone')
    print(new_rows)
    return new_rows

In [42]:
def clone_rows_active_vendor_facilty(input_df, column_to_change, new_values):
    """
    Clone rows in a DataFrame with Facility changed based on a list of values.
    Skip cloning if the new value matches the original row's condition.
    Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame.
    - column_to_change (str): The name of the column to be changed.
    - new_values (list): The list of new values for the specified column.

    Returns:
    - pd.DataFrame: The resulting DataFrame with cloned rows and the specified column changed.
    """
    # Step 1: Repeat the rows for each facility in new_values
    repeat_df = pd.concat([input_df] * len(new_values), ignore_index=True)

    # Step 2: Create a DataFrame with the repeated facility codes (new_values)
    new_values_df = pd.DataFrame({column_to_change: new_values * len(input_df)})

    # Step 3: Concatenate the new_values_df with the repeated DataFrame
    cloned_df = repeat_df.copy()
    cloned_df[column_to_change] = new_values_df[column_to_change].values

    # Step 4: Remove rows where the new value matches the original value (no need to clone identical rows)
    mask = cloned_df[column_to_change] != repeat_df[column_to_change]
    cloned_df = cloned_df[mask].reset_index(drop=True)
    repeat_df_filtered = repeat_df[mask].reset_index(drop=True)

    # Step 5: Add home_destination column to store the original facility code
    cloned_df['home_destination'] = repeat_df_filtered[column_to_change].values

    # Step 6: Update 'bic_vendor_type' column to 'C'
    cloned_df['bic_vendor_type'] = 'C'

    return cloned_df


In [44]:
%%timeit
result_df = clone_rows_active_vendor_facilty3(clone_df, 'bic_vendor_facility', facility_codes_list)
print('result_df',len(result_df))



result_df 799388
result_df 799388
result_df 799388
result_df 799388
result_df 799388
result_df 799388
result_df 799388
result_df 799388
43.2 s ± 2.62 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
result_df = clone_rows_active_vendor_facilty(clone_df, 'bic_vendor_facility', facility_codes_list)
print('result_df',len(result_df))



result_df 799388
CPU times: total: 43.7 s
Wall time: 1min 28s


In [38]:
result_df[['bic_vendor_type','home_destination','bic_vendor_facility']]

,bic_vendor_type,home_destination,bic_vendor_facility
0,C,14,305
1,C,14,93
2,C,14,403
3,C,14,404
4,C,14,241
...,...,...,...
799383,C,14,904
799384,C,14,905
799385,C,14,18
799386,C,14,255


In [39]:
def clone_rows_active_vendor_facilty3(input_df, column_to_change, new_values):
    """
    Clone rows in a DataFrame with Facility changed based on a list of values.
    Skip cloning if the new value matches the original row's condition.
    Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame.
    - column_to_change (str): The name of the column to be changed.
    - new_values (list): The list of new values for the specified column.

    Returns:
    - pd.DataFrame: The resulting DataFrame with cloned rows and the specified column changed.
    """
    # Step 1: Repeat the rows for each facility in new_values
    repeat_df = pd.concat([input_df] * len(new_values), ignore_index=True)

    # Step 2: Create a DataFrame with the repeated facility codes (new_values)
    new_values_df = pd.DataFrame({column_to_change: new_values * len(input_df)})

    # Step 3: Concatenate the new_values_df with the repeated DataFrame
    cloned_df = repeat_df.copy()
    cloned_df[column_to_change] = new_values_df[column_to_change].values

    # Step 4: Remove rows where the new value matches the original value (no need to clone identical rows)
    original_values = repeat_df[column_to_change].values
    cloned_df = cloned_df[cloned_df[column_to_change] != original_values].reset_index(drop=True)

    # Step 5: Add home_destination column to store the original facility code (only for cloned rows)
    cloned_df['home_destination'] = original_values[cloned_df.index]

    # Step 6: Update 'bic_vendor_type' column to 'C'
    cloned_df['bic_vendor_type'] = 'C'

    return cloned_df
